In [1]:
import numpy as np
import glob
import os
import random
from tqdm import tqdm
import gc


def memory(n):
    import psutil
    import os
    process = psutil.Process(os.getpid())
    print(str(n) + " : " + str(process.memory_info()[0]/1024/1024/1024))
    print(str(n) + " : " + str(psutil.virtual_memory()))


def mem_obj(obj):
    import sys
    print(str(sys.getsizeof(obj)/1024/1024/1024))


class Data_reader():
    # Override by subclasses

    def __init__(self, folder_name, np_shape, formats):
        self.folder_name = folder_name
        self.np_shape = np_shape
        self.formats = formats
        self.data = {}
        self.data['train'] = {}
        self.data['val'] = {}
        self.data['test'] = {}

    def __del__(self):
        del self.data

    # Search for all folder with name folder_name within the paths passed
    # Read all files inside using the read_file method and create a list
    # Finally, all file data is added to the data dictionary
    def read_data(self, paths, dataset='train'):
        for path in tqdm(paths):
            case_id = os.path.split(path)[-1]
            data_path = os.path.join(path, self.folder_name)
            if not os.path.exists(data_path) or len(os.listdir(data_path)) == 0:
                #pass
                self.data[dataset][case_id] = [np.full(self.np_shape,-3)]
            else:
                file_data = []
                for format in self.formats:
                    for file in glob.glob(data_path + "/*" + format):
                        file_data += [self.read_file(file)]
                self.data[dataset][case_id] = file_data

    def read_file(self, file):
        print("Read method not overwritten!")
        return None

    # Return a random data value for a case (if there are multiple)

    def get_data(self, mode, case_id):
        input_data = None
        if mode in self.data and case_id in self.data[mode]:
            input_data = self.data[mode][case_id]
            input_data = input_data[np.random.randint(0, len(input_data))]
        else:
            print("Error, {} or {} not present in data, returning None".format(mode, case_id))
        return input_data


In [6]:
paths = [""]

data = Data_reader("data", [1,1,1], [])

data.read_data(paths)

100%|██████████| 1/1 [00:00<00:00, 1846.90it/s]


In [8]:
data.get_data

<bound method Data_reader.get_data of <__main__.Data_reader object at 0x0000014D4CE04520>>

In [2]:
import large_image
import cv2

f = "TCGA-M8-A5N4-01A-01-TS1.E1110AEF-3812-4521-ACE9-17F70FBFF090.svs"

ts = large_image.getTileSource(f)

im_low_res , = ts.getRegion(
scale=dict(magnification =2.5),
format=large_image.tilesource.TILE_FORMAT_NUMPY)

Failed to loaded tile source openslide
Traceback (most recent call last):
  File "D:\Users\Alejandro\anaconda3\lib\site-packages\large_image\tilesource\__init__.py", line 60, in loadTileSources
    sourceClass = entryPoint.load()
  File "D:\Users\Alejandro\anaconda3\lib\importlib\metadata.py", line 77, in load
    module = import_module(match.group('module'))
  File "D:\Users\Alejandro\anaconda3\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "D:\Users\Alejandro\anaconda3\lib\site-pac

TileSourceFileNotFoundError: [Errno 2] TCGA-M8-A5N4-01A-01-TS1.E1110AEF-3812-4521-ACE9-17F70FBFF090.svs